In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

*회귀*는 가격이나 확률 같이 연속된 출력값을 예측하는 것이 목적이다. 이와 달리 분류는 여러개의 클래스 중 하나의 클래스를 선택하는 것이 목적이다. 

# Auto MPG 데이터셋

## 데이터 구하기

In [ ]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [ ]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']

                # na_values : 결측값으로 인식할 문자열 지정
                # https://yganalyst.github.io/data_handling/Pd_4/
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

## 데이터 정제하기

일부 누락된 데이터가 있다.

In [ ]:
dataset.isna().sum()

문제를 간단하게 하기 위해 누락된 행을 삭제한다.

In [ ]:
dataset = dataset.dropna()

'Origin'열은 수치형이 아니고 범주형이기 때문에 원-핫 인코딩으로 변환한다.

In [ ]:
origin = dataset.pop('Origin')

In [ ]:
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

## 데이터셋을 훈련 세트와 테스트 세트로 분할하기

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

## 데이터 조사하기
훈련 세트에서 몇개의 열을 선택해 산점도 행렬을 만든다.

In [ ]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

## 특성과 레이블 분리하기

특서에서 타깃값 또는 레이블을 분리한다. 이 레이블을 예측하기 위해 모델을 훈련시킬 것이다.

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

## 데이터 정규화

위 trian_stats 통게를 살펴보고 각 특성의 범위가 얼마나 다른지 확인해보다.

특성의 스케일과 범위가 다르면 정규화 하는 것이 권장된다. 특성을 정규화하지 않아도 모델이 수렴할 수 있지만, 훈련시키기 어렵고 입력 단위에 의존적인 모델이 만들어진다.



In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

# 모델

## 모델 만들기

여기에서는 두개의 Dense 은닉층으로 Sequential 모델을 만든다. 출력층은 하나의 연속적인 값을 반환한다. 나중에 두번째 모델을 만들기 쉽도록 build_model 함수로 모델 구성단계를 감싼다.

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

## 모델 확인

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

## 모델 훈련

이 모델을 1,000번의 에포크동안 훈련한다. 훈련 정확도와 검증 정확도는 history 객체에 기록된다.

In [ ]:
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure(figsize=(8,12))

  plt.subplot(2,1,1)
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.subplot(2,1,2)
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()

plot_history(history)

에포크를 반복 진행해도 모델이 향상되지 않는 것을 볼 수 있다. 그렇다면 model.fit 메서드를 수정하여 검증 점수가 향상되지 않으면 자동으로 훈련을 멎추어보자. 에포크마다 훈련 상태를 점검하기 위해 EarlyStopping callback을 사용하자. 

In [ ]:
model = build_model()

# patience 매개변수는 성능 향상을 체크할 에포크 횟수입니다
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("테스트 세트의 평균 절대 오차: {:5.2f} MPG".format(mae))

# 예측

마지막으로 테스트 세트에 있는 샘플을 사용해 MPG를 예측해 보자.

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
# 오차 분포

error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

# 결론

> 평균 제곱 오차 (MSE)는 회귀 문제에서 자주 사용하는 손실 함수이다.

> 비슷하게 회귀에서 사용되는 평가 지표도 분류와 다르다. 많이 사용하는 회귀 지표는 평균 절댓값 오차(MAE)이다. 

> 수치 입력 데이터의 특성이 여러가지 범위를 가질 때 동일한 범위가 되도록 각 특성의 스케일을 독립적으로 조정해야한다.

> 훈련 데이터가 많지 않다면 과대 적합을 피하기 위해 은닉층의 개수가 적은 소규모 네트워크를 선택하는 방법이 좋다.

> 조기 종료는 과대 적합을 방지하기 위해 좋은 방법이다.